In [3]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.ensemble
from sklearn.metrics         import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

import xgboost

import random

In [33]:
### Find cell types to predict for
mypath= "greta/feature"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles


['monocyte.csv',
 'astrocyte.csv',
 'pancreatic D cell.csv',
 'pancreatic A cell.csv',
 'blood cell.csv',
 'hepatocyte.csv',
 'macrophage.csv',
 'luminal epithelial cell of mammary gland.csv',
 'leukocyte.csv',
 'enteroendocrine cell.csv',
 'lung epithelial cell.csv',
 'ciliated cell.csv',
 'keratinocyte.csv',
 'stromal cell.csv',
 'myeloid cell.csv',
 'neuron.csv',
 'pancreatic PP cell.csv',
 'hematopoietic precursor cell.csv',
 'erythrocyte.csv',
 'pericyte.csv',
 'T cell.csv',
 'oligodendrocyte.csv',
 'endothelial cell.csv',
 'pancreatic ductal cell.csv',
 'B cell.csv',
 'type B pancreatic cell.csv',
 'common lymphoid progenitor.csv',
 'mesenchymal cell.csv',
 'muscle cell.csv',
 'kidney  epithelial cell.csv',
 'glial cell.csv',
 'endocardial cell.csv',
 'basophil.csv',
 'professional antigen presenting cell.csv',
 'myofibroblast cell.csv',
 'common myeloid progenitor.csv',
 'multipotent cell.csv',
 'pancreatic acinar cell.csv',
 'granulocyte.csv',
 'large intestine cell.csv',
 'epi

In [34]:
for fname in onlyfiles:
    print(fname)

    ######## Read data
    dataCSV = pd.read_csv(join(mypath, fname))   

    dataCSV = dataCSV.drop(columns=['rank_gwas_1', 'rank_tad', 'essentiality_ct'])

    dataCSV = dataCSV.sort_values(by='first_year')

    ######## Prepare test and train datasets
    data = pd.DataFrame(dataCSV)
    X_meta = data.iloc[:,0:5]
    X = data.iloc[:,6:]  #remove first descriptive cols
    y = data["rank_pmid"]  #Predict rank_pmid
    orig_year = data["orig_year"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


    # train XGBoost model
    xlf = xgboost.XGBRegressor(objective="reg:squarederror", seed=1729)
    xlf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])

    # calculate the auc score
    preds = xlf.predict(X_test)

    print('\nMean Square error" ', mean_squared_error(y_test,preds))

    #################################################################
    ##Function: Get the prediction from some data, as numpy array
    def predict_for(X):
        output = xlf.predict(X)
        return output


    #### Sensitivity analysis
    X_test_col = X_test.columns.tolist()
    y_test_predicted = predict_for(X_test)
    X_test_diff = X_test.copy()

    outlist=[]

    for i in range(0, len(X_test_col)):
        the_feature = X_test_col[i]
        #print(the_feature)
        X_test_perturb = X_test.copy()
        X_test_perturb[the_feature] = X_test_perturb + X_test[the_feature].std()
        y_test_perturbed_predicted = predict_for(X_test_perturb)

        X_test_diff[the_feature] = y_test_perturbed_predicted - y_test_predicted
        diff = ((y_test_perturbed_predicted - y_test_predicted)).mean()
        #print(diff)

        outlist.append(diff)


    df = pd.DataFrame(list(zip(X_test_col, outlist)), 
                   columns =['feature', 'val']) 
    df = df.sort_values(by="val")
    df.to_csv(join("plots/xgboost_w/",fname))
    df

    ## Store predictions
    y_all_predicted = predict_for(X)
    dictionary = dict(zip(X_meta["gene"].values.tolist(),y_all_predicted))
    df= pd.DataFrame.from_dict(dictionary, orient='index')

    df.to_csv(join("plots/xgboost_prediction/",fname))


monocyte.csv
[0]	validation_0-rmse:0.94855
[1]	validation_0-rmse:0.84732
[2]	validation_0-rmse:0.78796
[3]	validation_0-rmse:0.75367
[4]	validation_0-rmse:0.73241
[5]	validation_0-rmse:0.72354
[6]	validation_0-rmse:0.71665
[7]	validation_0-rmse:0.71429
[8]	validation_0-rmse:0.71211
[9]	validation_0-rmse:0.71172
[10]	validation_0-rmse:0.70913
[11]	validation_0-rmse:0.70746
[12]	validation_0-rmse:0.70744
[13]	validation_0-rmse:0.70668
[14]	validation_0-rmse:0.70644
[15]	validation_0-rmse:0.70655
[16]	validation_0-rmse:0.70624
[17]	validation_0-rmse:0.70683
[18]	validation_0-rmse:0.70804
[19]	validation_0-rmse:0.70835
[20]	validation_0-rmse:0.70929
[21]	validation_0-rmse:0.70870
[22]	validation_0-rmse:0.70856
[23]	validation_0-rmse:0.70957
[24]	validation_0-rmse:0.71011
[25]	validation_0-rmse:0.71049
[26]	validation_0-rmse:0.71065
[27]	validation_0-rmse:0.71081
[28]	validation_0-rmse:0.71127
[29]	validation_0-rmse:0.71225
[30]	validation_0-rmse:0.71235
[31]	validation_0-rmse:0.71248
[32]	

[62]	validation_0-rmse:0.71487
[63]	validation_0-rmse:0.71522
[64]	validation_0-rmse:0.71559
[65]	validation_0-rmse:0.71561
[66]	validation_0-rmse:0.71568
[67]	validation_0-rmse:0.71625
[68]	validation_0-rmse:0.71619
[69]	validation_0-rmse:0.71620
[70]	validation_0-rmse:0.71629
[71]	validation_0-rmse:0.71620
[72]	validation_0-rmse:0.71629
[73]	validation_0-rmse:0.71665
[74]	validation_0-rmse:0.71676
[75]	validation_0-rmse:0.71691
[76]	validation_0-rmse:0.71723
[77]	validation_0-rmse:0.71703
[78]	validation_0-rmse:0.71688
[79]	validation_0-rmse:0.71692
[80]	validation_0-rmse:0.71673
[81]	validation_0-rmse:0.71722
[82]	validation_0-rmse:0.71724
[83]	validation_0-rmse:0.71721
[84]	validation_0-rmse:0.71770
[85]	validation_0-rmse:0.71762
[86]	validation_0-rmse:0.71794
[87]	validation_0-rmse:0.71763
[88]	validation_0-rmse:0.71783
[89]	validation_0-rmse:0.71815
[90]	validation_0-rmse:0.71823
[91]	validation_0-rmse:0.71829
[92]	validation_0-rmse:0.71875
[93]	validation_0-rmse:0.71912
[94]	val

[22]	validation_0-rmse:0.71133
[23]	validation_0-rmse:0.71127
[24]	validation_0-rmse:0.71177
[25]	validation_0-rmse:0.71236
[26]	validation_0-rmse:0.71242
[27]	validation_0-rmse:0.71319
[28]	validation_0-rmse:0.71276
[29]	validation_0-rmse:0.71311
[30]	validation_0-rmse:0.71292
[31]	validation_0-rmse:0.71065
[32]	validation_0-rmse:0.71218
[33]	validation_0-rmse:0.71252
[34]	validation_0-rmse:0.71341
[35]	validation_0-rmse:0.71321
[36]	validation_0-rmse:0.71311
[37]	validation_0-rmse:0.71290
[38]	validation_0-rmse:0.71273
[39]	validation_0-rmse:0.71318
[40]	validation_0-rmse:0.71328
[41]	validation_0-rmse:0.71333
[42]	validation_0-rmse:0.71320
[43]	validation_0-rmse:0.71367
[44]	validation_0-rmse:0.71395
[45]	validation_0-rmse:0.71448
[46]	validation_0-rmse:0.71453
[47]	validation_0-rmse:0.71483
[48]	validation_0-rmse:0.71479
[49]	validation_0-rmse:0.71484
[50]	validation_0-rmse:0.71474
[51]	validation_0-rmse:0.71552
[52]	validation_0-rmse:0.71559
[53]	validation_0-rmse:0.71569
[54]	val

[83]	validation_0-rmse:0.71729
[84]	validation_0-rmse:0.71759
[85]	validation_0-rmse:0.71757
[86]	validation_0-rmse:0.71764
[87]	validation_0-rmse:0.71771
[88]	validation_0-rmse:0.71761
[89]	validation_0-rmse:0.71710
[90]	validation_0-rmse:0.71782
[91]	validation_0-rmse:0.71792
[92]	validation_0-rmse:0.71845
[93]	validation_0-rmse:0.71842
[94]	validation_0-rmse:0.71870
[95]	validation_0-rmse:0.71836
[96]	validation_0-rmse:0.71859
[97]	validation_0-rmse:0.71897
[98]	validation_0-rmse:0.71908
[99]	validation_0-rmse:0.72004

Mean Square error"  0.5184549856180811
leukocyte.csv
[0]	validation_0-rmse:0.94974
[1]	validation_0-rmse:0.84669
[2]	validation_0-rmse:0.78649
[3]	validation_0-rmse:0.75537
[4]	validation_0-rmse:0.73850
[5]	validation_0-rmse:0.72826
[6]	validation_0-rmse:0.72123
[7]	validation_0-rmse:0.71821
[8]	validation_0-rmse:0.71428
[9]	validation_0-rmse:0.71211
[10]	validation_0-rmse:0.71174
[11]	validation_0-rmse:0.70941
[12]	validation_0-rmse:0.70836
[13]	validation_0-rmse:0.7

[43]	validation_0-rmse:0.70799
[44]	validation_0-rmse:0.70799
[45]	validation_0-rmse:0.70843
[46]	validation_0-rmse:0.70823
[47]	validation_0-rmse:0.70812
[48]	validation_0-rmse:0.70835
[49]	validation_0-rmse:0.70846
[50]	validation_0-rmse:0.70856
[51]	validation_0-rmse:0.70834
[52]	validation_0-rmse:0.70820
[53]	validation_0-rmse:0.70773
[54]	validation_0-rmse:0.70791
[55]	validation_0-rmse:0.70825
[56]	validation_0-rmse:0.70823
[57]	validation_0-rmse:0.70838
[58]	validation_0-rmse:0.70853
[59]	validation_0-rmse:0.70841
[60]	validation_0-rmse:0.70862
[61]	validation_0-rmse:0.70919
[62]	validation_0-rmse:0.70919
[63]	validation_0-rmse:0.70988
[64]	validation_0-rmse:0.71002
[65]	validation_0-rmse:0.70940
[66]	validation_0-rmse:0.70932
[67]	validation_0-rmse:0.70927
[68]	validation_0-rmse:0.70932
[69]	validation_0-rmse:0.70972
[70]	validation_0-rmse:0.71049
[71]	validation_0-rmse:0.71073
[72]	validation_0-rmse:0.71121
[73]	validation_0-rmse:0.71122
[74]	validation_0-rmse:0.71138
[75]	val

...

[49]	validation_0-rmse:0.70953
[50]	validation_0-rmse:0.70947
[51]	validation_0-rmse:0.71039
[52]	validation_0-rmse:0.71043
[53]	validation_0-rmse:0.71030
[54]	validation_0-rmse:0.71022
[55]	validation_0-rmse:0.71044
[56]	validation_0-rmse:0.71046
[57]	validation_0-rmse:0.71009
[58]	validation_0-rmse:0.71066
[59]	validation_0-rmse:0.71033
[60]	validation_0-rmse:0.71066
[61]	validation_0-rmse:0.71083
[62]	validation_0-rmse:0.71125
[63]	validation_0-rmse:0.71240
[64]	validation_0-rmse:0.71254
[65]	validation_0-rmse:0.71221
[66]	validation_0-rmse:0.71214
[67]	validation_0-rmse:0.71252
[68]	validation_0-rmse:0.71248
[69]	validation_0-rmse:0.71239
[70]	validation_0-rmse:0.71249
[71]	validation_0-rmse:0.71257
[72]	validation_0-rmse:0.71193
[73]	validation_0-rmse:0.71230
[74]	validation_0-rmse:0.71273
[75]	validation_0-rmse:0.71304
[76]	validation_0-rmse:0.71350
[77]	validation_0-rmse:0.71389
[78]	validation_0-rmse:0.71374
[79]	validation_0-rmse:0.71395
[80]	validation_0-rmse:0.71387
[81]	val

[9]	validation_0-rmse:0.71210
[10]	validation_0-rmse:0.71032
[11]	validation_0-rmse:0.70806
[12]	validation_0-rmse:0.70776
[13]	validation_0-rmse:0.70854
[14]	validation_0-rmse:0.70855
[15]	validation_0-rmse:0.70801
[16]	validation_0-rmse:0.70860
[17]	validation_0-rmse:0.70911
[18]	validation_0-rmse:0.70924
[19]	validation_0-rmse:0.70903
[20]	validation_0-rmse:0.70873
[21]	validation_0-rmse:0.70883
[22]	validation_0-rmse:0.70924
[23]	validation_0-rmse:0.70875
[24]	validation_0-rmse:0.70913
[25]	validation_0-rmse:0.70905
[26]	validation_0-rmse:0.70979
[27]	validation_0-rmse:0.71005
[28]	validation_0-rmse:0.70985
[29]	validation_0-rmse:0.71020
[30]	validation_0-rmse:0.71106
[31]	validation_0-rmse:0.71080
[32]	validation_0-rmse:0.71015
[33]	validation_0-rmse:0.70953
[34]	validation_0-rmse:0.70968
[35]	validation_0-rmse:0.70890
[36]	validation_0-rmse:0.70913
[37]	validation_0-rmse:0.70953
[38]	validation_0-rmse:0.70981
[39]	validation_0-rmse:0.71013
[40]	validation_0-rmse:0.71083
[41]	vali

[71]	validation_0-rmse:0.70906
[72]	validation_0-rmse:0.70929
[73]	validation_0-rmse:0.70895
[74]	validation_0-rmse:0.70892
[75]	validation_0-rmse:0.70895
[76]	validation_0-rmse:0.70902
[77]	validation_0-rmse:0.70904
[78]	validation_0-rmse:0.70904
[79]	validation_0-rmse:0.70918
[80]	validation_0-rmse:0.70870
[81]	validation_0-rmse:0.70865
[82]	validation_0-rmse:0.70865
[83]	validation_0-rmse:0.70905
[84]	validation_0-rmse:0.70966
[85]	validation_0-rmse:0.70928
[86]	validation_0-rmse:0.70983
[87]	validation_0-rmse:0.70964
[88]	validation_0-rmse:0.70990
[89]	validation_0-rmse:0.70984
[90]	validation_0-rmse:0.70973
[91]	validation_0-rmse:0.70969
[92]	validation_0-rmse:0.70768
[93]	validation_0-rmse:0.70812
[94]	validation_0-rmse:0.70840
[95]	validation_0-rmse:0.70852
[96]	validation_0-rmse:0.70886
[97]	validation_0-rmse:0.70875
[98]	validation_0-rmse:0.70862
[99]	validation_0-rmse:0.70892

Mean Square error"  0.5025723353132496
fibroblast.csv
[0]	validation_0-rmse:0.94868
[1]	validation_0

[31]	validation_0-rmse:0.70686
[32]	validation_0-rmse:0.70658
[33]	validation_0-rmse:0.70667
[34]	validation_0-rmse:0.70693
[35]	validation_0-rmse:0.70800
[36]	validation_0-rmse:0.70843
[37]	validation_0-rmse:0.70851
[38]	validation_0-rmse:0.70857
[39]	validation_0-rmse:0.70846
[40]	validation_0-rmse:0.70859
[41]	validation_0-rmse:0.70907
[42]	validation_0-rmse:0.70906
[43]	validation_0-rmse:0.70921
[44]	validation_0-rmse:0.70863
[45]	validation_0-rmse:0.70749
[46]	validation_0-rmse:0.70769
[47]	validation_0-rmse:0.70804
[48]	validation_0-rmse:0.70843
[49]	validation_0-rmse:0.70887
[50]	validation_0-rmse:0.70920
[51]	validation_0-rmse:0.70907
[52]	validation_0-rmse:0.70910
[53]	validation_0-rmse:0.70930
[54]	validation_0-rmse:0.70926
[55]	validation_0-rmse:0.70958
[56]	validation_0-rmse:0.70953
[57]	validation_0-rmse:0.70985
[58]	validation_0-rmse:0.71066
[59]	validation_0-rmse:0.71108
[60]	validation_0-rmse:0.71057
[61]	validation_0-rmse:0.71033
[62]	validation_0-rmse:0.71049
[63]	val

[93]	validation_0-rmse:0.72380
[94]	validation_0-rmse:0.72412
[95]	validation_0-rmse:0.72439
[96]	validation_0-rmse:0.72443
[97]	validation_0-rmse:0.72456
[98]	validation_0-rmse:0.72463
[99]	validation_0-rmse:0.72452

Mean Square error"  0.5249343034884386
basal cell.csv
[0]	validation_0-rmse:0.94909
[1]	validation_0-rmse:0.84850
[2]	validation_0-rmse:0.78731
[3]	validation_0-rmse:0.75521
[4]	validation_0-rmse:0.73877
[5]	validation_0-rmse:0.72830
[6]	validation_0-rmse:0.71948
[7]	validation_0-rmse:0.71598
[8]	validation_0-rmse:0.71363
[9]	validation_0-rmse:0.71195
[10]	validation_0-rmse:0.71060
[11]	validation_0-rmse:0.70944
[12]	validation_0-rmse:0.70823
[13]	validation_0-rmse:0.70532
[14]	validation_0-rmse:0.70399
[15]	validation_0-rmse:0.70406
[16]	validation_0-rmse:0.70331
[17]	validation_0-rmse:0.70386
[18]	validation_0-rmse:0.70317
[19]	validation_0-rmse:0.70387
[20]	validation_0-rmse:0.70398
[21]	validation_0-rmse:0.70421
[22]	validation_0-rmse:0.70427
[23]	validation_0-rmse:0.